### Function Creation and Logical Tests

* This is a notebook to create the functions for the simulations and for small tests of the implemented logic.

##### Calculating the Time to Complete AP for each edge

In [ ]:
def PosArray_to_SignalTiming(
    pos_array,
    edges,
    config_dict,
):
    '''
    A function to go from the pos_array to the telegrapher's outputs.
    
    Inputs:
        self: The OrganoidEnv object.
    
    '''

    ## Put the parameters into the function name space instead of constantly indexing the dictinoary
    # Timing
    num_steps = config_dict['timing']['num_steps']
    t_step_size = config_dict['timing']['t_step_size']
    noise_injected_current_amount = config_dict['timing']['noise_injected_current_amount']

    # Electical parameters
    rushton_constant = config_dict['electrical-parameters']['rushton_constant'] 
    resistivity = config_dict['electrical-parameters']['resistivity'] 
    e_myelin = config_dict['electrical-parameters']['e_myelin'] 
    inductance = config_dict['electrical-parameters']['inductance'] 
    e_lipid = config_dict['electrical-parameters']['e_lipid'] 
    lipid_width = config_dict['electrical-parameters']['lipid_width'] 
    t_max_divisor = config_dict['electrical-parameters']['t_max_divisor'] 
    allometric_scaling_exponent = config_dict['electrical-parameters']['allometric_scaling_exponent'] 
    alpha_divisor = config_dict['electrical-parameters']['alpha_divisor'] 
    beta_divisor = config_dict['electrical-parameters']['beta_divisor'] 
    initial_signal_amplitude = config_dict['electrical-parameters']['initial_signal_amplitude'] 
    membrane_time_constant = config_dict['electrical-parameters']['membrane_time_constant'] 

    # G-Ratio
    unmyelinated_gratio = config_dict['g-ratio']['unmyelinated_gratio']
    gratio_sd = config_dict['g-ratio']['gratio_sd']
    gratio_lower_bound = config_dict['g-ratio']['gratio_lower_bound']
    theta_scalar_myelin = config_dict['g-ratio']['theta_scalar_myelin']
    theta_scalar_non_myelin = config_dict['g-ratio']['theta_scalar_non_myelin']
    g_ratio_diameter_threshold = config_dict['g-ratio']['g_ratio_diameter_threshold']
    gratio_threshold = config_dict['g-ratio']['gratio_threshold']
    
    # Constants defined in the function
    gratio_mean = np.exp(-1/2) # ~0.6 from Rushton
    num_edges = edges.shape[0]

    # Create a distribution for G-Ratios
    gratio_distribution = stats.truncnorm(
        (gratio_lower_bound - gratio_mean) / gratio_sd,
        (unmyelinated_gratio - gratio_mean) / gratio_sd,
        loc = gratio_mean, 
        scale = gratio_sd
    )

    gratio_distribution = stats.truncnorm(
        (gratio_lower_bound - gratio_mean) / gratio_sd,
        (unmyelinated_gratio - gratio_mean) / gratio_sd,
        loc = gratio_mean, 
        scale = gratio_sd
    )

    # Randomly draw g-ratios based on the 
    g_ratios = np.array(gratio_distribution.rvs(num_edges), dtype = np.float64)
    
    # Calculate the length of each axon
    distances = np.linalg.norm(
        pos_array[edges[:, 0], :] - pos_array[edges[:, 1], :], 
        axis = 1
    )
    
    # Defining the inner diameter of the neuron (d)
    d = distances**allometric_scaling_exponent
    
    myelinated_index = (d >= gratio_threshold)
    
    unmyelinated_index = (1 - myelinated_index)

    # Calculating how far the g-ratios for unmyelinated axons are from the desired g-ratio for unmyelinated neurons
    g_ratios[unmyelinated_index] = unmyelinated_gratio
    
    # Defining the outer diameter
    # If it's myelinated, it should be the same as d. 
    D = (d / g_ratios)

    # Calculate the myelin lengths as a mix of the formula for the myelinated vs unmyelinated
    myelin_lengths = ( 
        myelinated_index * (d * rushton_constant * np.sqrt(-np.log(g_ratios))) # If myelinated, calculate this way
        + unmyelinated_index * (distances) # If unmyelinated, calcuate as just the whole distance of the axon
    )

    ### Calculate the value for theta, scale by the time step size so it's an actual speed value
    thetas = (
        myelinated_index * (d * np.sqrt(-np.log(g_ratios)) ) * (theta_scalar_myelin / t_step_size) # Myelinated calculation for theta from Rushton
        + unmyelinated_index * (np.sqrt(d) * (theta_scalar_non_myelin / t_step_size)) # Unmyelinated calculation for theta
    )
    
    # Values for t_max and total
    t_max = myelin_lengths / thetas
    time_to_complete_ap = distances / thetas

    return time_to_complete_ap

